In [41]:
import cv2
import argparse
import numpy as np 
from glob import glob
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import precision_recall_fscore_support as prfs


In [42]:
# noOfSamples
noOfClasses=19
testPath = './../fics-logoaugmentator/split/val/'
trainPath = './../fics-logoaugmentator/split/train/'
imgSize= 100


In [43]:
class ImageHelpers:
	def __init__(self):
		self.sift_object = cv2.xfeatures2d.SIFT_create()
	
	def scale(im):
		return cv2.resize(im, (imgSize,imgSize))
		
	def gray(self, image):
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
		return scale(gray)

	def features(self, image):
		keypoints, descriptors = self.sift_object.detectAndCompute(image, None)
		return [keypoints, descriptors]

In [44]:
class BOVHelpers:
	def __init__(self, n_clusters = noOfClasses):
		self.n_clusters = n_clusters
		self.kmeans_obj = KMeans(n_clusters = n_clusters)
		self.kmeans_ret = None
		self.descriptor_vstack = None
		self.mega_histogram = None
		self.clf  = SVC()	

	def cluster(self):
		"""	
		cluster using KMeans algorithm, 
		"""
		self.kmeans_ret = self.kmeans_obj.fit_predict(self.descriptor_vstack)

	def developVocabulary(self,n_images, descriptor_list, kmeans_ret = None):
		
		"""
		Each cluster denotes a particular visual word 
		Every image can be represeted as a combination of multiple 
		visual words. The best method is to generate a sparse histogram
		that contains the frequency of occurence of each visual word 
		Thus the vocabulary comprises of a set of histograms of encompassing
		all descriptions for all images
		"""

		self.mega_histogram = np.array([np.zeros(self.n_clusters) for i in range(n_images)])
		old_count = 0
		for i in range(len(descriptor_list)):
			l = len(descriptor_list[i])
			for j in range(l):
				if kmeans_ret is None:
					idx = self.kmeans_ret[old_count+j]
				else:
					idx = kmeans_ret[old_count+j]
				self.mega_histogram[i][idx] += 1
			old_count += l
		print("Vocabulary Histogram Generated")

	def standardize(self, std=None):
		"""
		
		standardize is required to normalize the distribution
		wrt sample size and features. If not normalized, the classifier may become
		biased due to steep variances.
		"""
		if std is None:
			self.scale = StandardScaler().fit(self.mega_histogram)
			self.mega_histogram = self.scale.transform(self.mega_histogram)
		else:
			print("STD not none. External STD supplied")
			self.mega_histogram = std.transform(self.mega_histogram)

	def formatND(self, l):
		"""	
		restructures list into vstack array of shape
		M samples x N features for sklearn
		"""
		vStack = np.array(l[0])
		for remaining in l[1:]:
			vStack = np.vstack((vStack, remaining))
		self.descriptor_vstack = vStack.copy()
		return vStack

	def train(self, train_labels):
		"""
		uses sklearn.svm.SVC classifier (SVM) 
		"""
		print("Training SVM")
		print (self.clf)
		print("Train labels", train_labels)
		self.clf.fit(self.mega_histogram, train_labels)
		print ("Training completed")

	def predict(self, iplist):
		predictions = self.clf.predict(iplist)
		return predictions

	def plotHist(self, vocabulary = None):
		print ("Plotting histogram")
		if vocabulary is None:
			vocabulary = self.mega_histogram

		x_scalar = np.arange(self.n_clusters)
		y_scalar = np.array([abs(np.sum(vocabulary[:,h], dtype=np.int32)) for h in range(self.n_clusters)])

		# print (y_scalar)

		plt.bar(x_scalar, y_scalar)
		plt.xlabel("Visual Word Index")
		plt.ylabel("Frequency")
		plt.title("Complete Vocabulary Generated")
		plt.xticks(x_scalar + 0.4, x_scalar)
		plt.show()

In [45]:
class FileHelpers:

	def __init__(self):
		pass

	def getFiles(self, path):
		"""
		- returns  a dictionary of all files 
		having key => value as  objectname => image path
		- returns total number of files.
		"""
		imlist = {}
		classCount,count = 0,0
		for each in glob(path + "*"):
			word = (each.split("\\")[-1]).strip()
			print(" #### Reading image category", word, " ##### ")
			imlist[word] = []
			# print(path+word+"\*")
			for imagefile in glob(path+word+"\*"):
				# print ("Reading file ", imagefile)
				im = cv2.imread(imagefile, 0)
# 				print(im.shape)
# 				plt.figure()
# 				plt.imshow(im,'gray')
				imlist[word].append(im)
				count +=1 
			classCount +=1 
		return [imlist, count]

In [46]:
# test_fh = FileHelpers()
# imglist,count = test_fh.getFiles(testPath)

In [47]:
# print(len(imglist),count)
# imgll=[len(k) for k in imglist.values()]
# print(min(imgll))

In [48]:
class BOV:
    def __init__(self, no_clusters):
        self.no_clusters = no_clusters
        self.train_path = None
        self.test_path = None
        self.im_helper = ImageHelpers()
        # self.bov_helper = BOVHelpers()
        self.bov_helper = BOVHelpers(no_clusters)
        
        self.file_helper = FileHelpers()
        self.images = None
        self.trainImageCount = 0
        self.train_labels = np.array([])
        self.name_dict = {}
        self.descriptor_list = []

    def trainModel(self):
        """
        This method contains the entire module 
        required for training the bag of visual words model
        Use of helper functions will be extensive.
        """

        # read file. prepare file lists.
        self.images, self.trainImageCount = self.file_helper.getFiles(self.train_path)
        # extract SIFT Features from each image
        label_count = 0 
        for word, imlist in self.images.items():
            self.name_dict[str(label_count)] = word
            print("Computing Features for ", word)
            for im in imlist:
        
                self.train_labels = np.append(self.train_labels, label_count)
                kp, des = self.im_helper.features(im)
                if des is not None:
                    self.descriptor_list.append(des)

            label_count += 1


        # perform clustering
        bov_descriptor_stack = self.bov_helper.formatND(self.descriptor_list)
        self.bov_helper.cluster()
        self.bov_helper.developVocabulary(n_images = self.trainImageCount, descriptor_list=self.descriptor_list)

        # show vocabulary trained
        self.bov_helper.plotHist()
 

        self.bov_helper.standardize()
        self.bov_helper.train(self.train_labels)


    def recognize(self,test_img, test_image_path=None):

        """ 
        This method recognizes a single image 
        It can be utilized individually as well.
        """

        kp, des = self.im_helper.features(test_img)
        # print kp
        # print(des.shape)

        # generate vocab for test image
        vocab = np.array( [[ 0 for i in range(noOfClasses)]])
        # locate nearest clusters for each of 
        # the visual word (feature) present in the image
        
        # test_ret =<> return of kmeans nearest clusters for N features
        test_ret = self.bov_helper.kmeans_obj.predict(des)
        # print(test_ret)

        # print vocab
        for each in test_ret:
            # if each<18:
                vocab[0][each] += 1

        # print(vocab)
        # Scale the features
        vocab = self.bov_helper.scale.transform(vocab)

        # predict the class of the image
        lb = self.bov_helper.clf.predict(vocab)
        # print("Image belongs to class : ", self.name_dict[str(int(lb[0]))])
        return lb



    def testModel(self):
        """ 
        This method is to test the trained classifier
        read all images from testing path 
        use BOVHelpers.predict() function to obtain classes of each image
        """
        print("Using Test Path:",self.test_path)
        self.testImages, self.testImageCount = self.file_helper.getFiles(self.test_path)

        predictions = []
        y_test,y_pred=[],[]
        label_count,total,correct=0,0,0
        for word, imlist in self.testImages.items():
            print("processing ..." ,word)
            for im in imlist:
                # print imlist[0].shape, imlist[1].shape
                # print(im.shape)
                if im is not None:
                    cl = self.recognize(im)
                    # print(cl[0])
                    predictions.append({
                        'image':im,
                        'class':cl,
                        'object_name':self.name_dict[str(int(cl[0]))],
                        'real_name':self.name_dict[str(int(label_count))]                        
                        })
                    y_test+=[label_count]
                    y_pred+=[cl[0]]
                    total+=1
                    if cl[0] == label_count:
                        correct+=1
            label_count += 1

        # print(predictions)
        # for each in predictions:
        #     cv2.imshow(each['real_name']+"->"+each['object_name'], each['image'])
        #     cv2.waitKey()
        #     cv2.destroyWindow(each['object_name'])
            
        #     plt.imshow(cv2.cvtColor(each['image'], cv2.COLOR_GRAY2RGB))
        #     plt.title(each['object_name'])
        #     plt.show()
        precision, recall, f1, support = prfs(y_test, y_pred, average='weighted')
        print("Precision: {:.2%}\nRecall: {:.2%}\nF1 score: {:.2%}".format(precision, recall, f1))
        print("Accuracy :",(correct/total)*100,"%")


    def print_vars(self):
        pass


In [49]:
bov = BOV(noOfClasses)
# set training paths
bov.train_path = trainPath


In [50]:
bov.trainModel()

 #### Reading image category Altera  ##### 
 #### Reading image category Analog Devices  ##### 
 #### Reading image category Bel fuse inc  ##### 
 #### Reading image category Fairchild Semiconductor  ##### 
 #### Reading image category General Semiconductors Industries Inc  ##### 
 #### Reading image category Holtek Semiconductors  ##### 
 #### Reading image category Level One  ##### 
 #### Reading image category Linear Technologies  ##### 
 #### Reading image category Micron Technologies  ##### 
 #### Reading image category Mitsubishi Electric Corporation  ##### 
 #### Reading image category Motorola Semiconductor Products Inc  ##### 
 #### Reading image category Nvidia  ##### 
 #### Reading image category ON Semiconductors  ##### 
 #### Reading image category Pericom Semiconductors  ##### 
 #### Reading image category Realtek Semiconductors  ##### 
 #### Reading image category Recongized Component Mark  ##### 
 #### Reading image category STMicroelectronics  ##### 
 #### Reading imag

error: OpenCV(4.5.1) c:\users\appveyor\appdata\local\temp\1\pip-req-build-i1s8y2i1\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x67dbbff5::Set<1,-1,-1>,struct cv::impl::A0x67dbbff5::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [ ]:
# set testing paths
print(testPath)
bov.test_path = testPath
bov.testModel()